In [1]:
%load_ext autoreload
%autoreload 2

import os

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')

from utilities2014 import *

import time

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from scipy.spatial.distance import cdist, pdist, squareform
from scipy.cluster.hierarchy import average, fcluster, leaders, complete, single, dendrogram, ward

from joblib import Parallel, delayed

from skimage.color import gray2rgb
from skimage.util import img_as_float, pad
from skimage.morphology import disk
from skimage.filters.rank import gradient

from collections import defaultdict, Counter
from itertools import combinations, chain, product

import networkx
from networkx import from_dict_of_lists, dfs_postorder_nodes

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
thetas = np.linspace(-np.pi/4, np.pi/4, 9)
n_theta = len(thetas)

In [3]:
dms = [DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=i) for i in range(7)]
for dm in dms:
    dm._load_image()

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [5]:
lms = []

for lm_ind in range(6):

        lm_texture_template_rotated_versions = [np.load('/home/yuncong/csd395/lm_texture_template_%d_orientation%d.npy'%(lm_ind, theta_i)) 
                                                    for theta_i in range(n_theta)]
        lm_bbox_dims_rotated_versions = [np.load('/home/yuncong/csd395/lm_bbox_dims_%d_orientation%d.npy'%(lm_ind, theta_i)) 
                                                for theta_i in range(n_theta)]
        lm_edges_tuple_rotated_versions = [np.load('/home/yuncong/csd395/lm_edge_points_%d_orientation%d.npy'%(lm_ind, theta_i)) 
                                                for theta_i in range(n_theta)]
        
        lms.append((lm_texture_template_rotated_versions, 
                    lm_bbox_dims_rotated_versions,
                   lm_edges_tuple_rotated_versions))

In [13]:
def visualize_top_landmark_responses(section_ind, lm_ind):
    dm = dms[section_ind]
    
    lm_texture_template_rotated_versions, lm_bbox_dims_rotated_versions, lm_edges_tuple_rotated_versions = lms[lm_ind]

    bg_ubyte = img_as_ubyte(dm.image_rgb)

    top3_locs = dm.load_pipeline_result('responsePeaksLm%d'%lm_ind, 'npy')

    for i, (y, x, v, a) in enumerate(top3_locs[:3]):
#         print v, a

        shifted_ys = (y - lm_bbox_dims_rotated_versions[int(a)][9] + lm_edges_tuple_rotated_versions[int(a)][:,3]).astype(np.int)
        shifted_xs = (x - lm_bbox_dims_rotated_versions[int(a)][8] + lm_edges_tuple_rotated_versions[int(a)][:,2]).astype(np.int)

        overlay = np.zeros_like(dm.image, np.bool)
        overlay[shifted_ys, shifted_xs] = 1
        overlay = binary_dilation(overlay, disk(5))
        c = np.random.randint(0,255,3)
        bg_ubyte[overlay] = c

    #     print shifted_xmean_best.astype(np.int), shifted_ymean_best.astype(np.int)

        cv2.putText(bg_ubyte, str(i), (x.astype(np.int), y.astype(np.int)), 
                    cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 3, (c), 10)

    return bg_ubyte

In [14]:
display(visualize_top_landmark_responses(2,2))

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [17]:
def visualize_and_save(section_ind, lm_ind): 
    viz = visualize_top_landmark_responses(section_ind, lm_ind)
    dms[section_ind].save_pipeline_result(viz, 'topLocsLm%d'%lm_ind, 'jpg')

_ = Parallel(n_jobs=16)(delayed(visualize_and_save)(section_ind, lm_ind) 
                for section_ind, lm_ind in product([0]+range(2,7), range(6)))

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm5.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm2.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm1.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm3.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm4.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm0.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014resu

In [ ]:
plt.matshow(vs_max_smooth);
plt.colorbar();